In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np

path = '../data/TSLA_10y.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume,Dividends,Stock Splits
0,2014-02-24 05:00:00+00:00,13.917333,14.557333,13.888000,14.510000,124176000,0.0,0.0
1,2014-02-25 05:00:00+00:00,15.333333,17.280001,15.230000,16.533333,490225500,0.0,0.0
2,2014-02-26 05:00:00+00:00,17.238667,17.666668,16.500000,16.866667,369069000,0.0,0.0
3,2014-02-27 05:00:00+00:00,17.416668,17.459999,16.555332,16.836000,269187000,0.0,0.0
4,2014-02-28 05:00:00+00:00,16.643333,16.845333,16.170000,16.320667,218847000,0.0,0.0


In [2]:
temp = df.ta.bbands()
df['BBANDS_U'] = temp['BBU_5_2.0']
df['BBANDS_L'] = temp['BBL_5_2.0']
df['BBANDS_M'] = temp['BBM_5_2.0']
df['DC_L'] = df.ta.donchian()['DCL_20_20']
df['DC_U'] = df.ta.donchian()['DCU_20_20']
df['SMA_20'] = df.ta.sma(length=20)
df['SMA_50'] = df.ta.sma(length=50)
df['SMA_200'] = df.ta.sma(length=200)
temp = df.ta.cdl_pattern(name="closingmarubozu")
df['MARUBOZU'] = temp['CDL_CLOSINGMARUBOZU'].replace(100,1).replace(-100,-1)
#singal_buy = MARUBOZU == 100 and close > DONCH_U 
#singal_sell = MARUBOZU == -100 and close < DONCH_L
df['Price_above_SMA_20'] = np.where(df['close'] > df['SMA_20'], 1, 0)
df['Price_above_SMA_50'] = np.where(df['close'] > df['SMA_50'], 1, 0)
df['Price_above_SMA_200'] = np.where(df['close'] > df['SMA_200'], 1, 0)
df['Signal_Buy'] = np.where((df['close'] > df['BBANDS_L']) & (df['close'] < df['BBANDS_M']), 1, 0)
df['Signal_Sell'] = np.where((df['close'] < df['BBANDS_U']) & (df['close'] > df['BBANDS_M']), 1, 0)
df['Signal'] = df['Signal_Buy'].replace(1,2) + df['Signal_Sell'].replace(1,3)
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [3]:
df['Signal'].value_counts() , df['Signal_Buy'].value_counts(), df['Signal_Sell'].value_counts()

(Signal
 3    1342
 2    1170
 0       4
 Name: count, dtype: int64,
 Signal_Buy
 0    1346
 1    1170
 Name: count, dtype: int64,
 Signal_Sell
 1    1342
 0    1174
 Name: count, dtype: int64)

In [4]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,Dividends,Stock Splits,BBANDS_U,BBANDS_L,...,SMA_200,MARUBOZU,Price_above_SMA_20,Price_above_SMA_50,Price_above_SMA_200,Signal_Buy,Signal_Sell,Signal,ATR,ADX
199,2014-12-05 05:00:00+00:00,15.244667,15.292667,14.817333,14.914000,90954000,0.0,0.0,15.641731,14.874536,...,15.489737,0.0,0,0,0,1,0,2,0.507733,22.029153
200,2014-12-08 05:00:00+00:00,14.769333,14.990667,14.156000,14.290667,138384000,0.0,0.0,15.837895,14.217572,...,15.488640,0.0,0,0,0,1,0,2,0.531085,24.311505
201,2014-12-09 05:00:00+00:00,13.956000,14.515333,13.618000,14.459333,141472500,0.0,0.0,15.631294,14.036440,...,15.478270,0.0,0,0,0,1,0,2,0.557246,26.868579
202,2014-12-10 05:00:00+00:00,14.275333,14.451333,13.846667,13.989333,109711500,0.0,0.0,15.453698,13.695102,...,15.463883,0.0,0,0,0,1,0,2,0.561204,29.243005
203,2014-12-11 05:00:00+00:00,14.035333,14.362000,13.882000,13.925333,100416000,0.0,0.0,15.030224,13.601242,...,15.449330,0.0,0,0,0,1,0,2,0.555404,31.447829


In [5]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_17160\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,Dividends,Stock Splits,BBANDS_U,BBANDS_L,BBANDS_M,...,SMA_200,MARUBOZU,Price_above_SMA_20,Price_above_SMA_50,Price_above_SMA_200,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,,,,,,,,
2014-12-05 05:00:00+00:00,15.244667,15.292667,14.817333,14.914000,90954000,0.0,0.0,15.641731,14.874536,15.258134,...,15.489737,0.0,0,0,0,1,0,2,0.507733,22.029153
2014-12-08 05:00:00+00:00,14.769333,14.990667,14.156000,14.290667,138384000,0.0,0.0,15.837895,14.217572,15.027733,...,15.488640,0.0,0,0,0,1,0,2,0.531085,24.311505
2014-12-09 05:00:00+00:00,13.956000,14.515333,13.618000,14.459333,141472500,0.0,0.0,15.631294,14.036440,14.833867,...,15.478270,0.0,0,0,0,1,0,2,0.557246,26.868579
2014-12-10 05:00:00+00:00,14.275333,14.451333,13.846667,13.989333,109711500,0.0,0.0,15.453698,13.695102,14.574400,...,15.463883,0.0,0,0,0,1,0,2,0.561204,29.243005
2014-12-11 05:00:00+00:00,14.035333,14.362000,13.882000,13.925333,100416000,0.0,0.0,15.030224,13.601242,14.315733,...,15.449330,0.0,0,0,0,1,0,2,0.555404,31.447829


In [6]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [7]:
df['Signal'].value_counts()

Signal
3    1235
2    1082
Name: count, dtype: int64

In [8]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Weight_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 10000
    profit_target = 12000
    size_multiplier_rate = 0.5
    kwargs = {}

    def init(self, **kwargs):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')
        self.lp1, self.lp2, self.lp3, self.lv1 = self.kwargs

    def next(self):
        super().next()
        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0
        buy_confidence = (self.lp1 * self.data.MARUBOZU[-1] + self.lp2 * self.data.Price_above_SMA_20[-1] + self.lp3 * self.data.Price_above_SMA_50[-1])

        if self.signal == 2 and buy_confidence >= self.lv1:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)
            
cash = 10000
bt = Backtest(df, Weight_Strategy, cash=cash, commission=.00002, margin=.05)
stat = bt.run(kwargs=[0.5, 0.3, 0.2, 0.5])
stat

Start                     2014-12-05 05:00...
End                       2024-02-21 05:00...
Duration                   3365 days 00:00:00
Exposure Time [%]                    0.129478
Equity Final [$]                          0.0
Equity Peak [$]                       10000.0
Return [%]                             -100.0
Buy & Hold Return [%]             1205.954204
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]             9049.374215
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                      -100.0
Max. Drawdown Duration     3299 days 00:00:00
Avg. Drawdown Duration     3299 days 00:00:00
# Trades                                    2
Win Rate [%]                              0.0
Best Trade [%]                      -4.336058
Worst Trade [%]                     -8.785573
Avg. Trade [%]                    

In [9]:
import torch
from tqdm import trange

parm_tensor = torch.tensor([0.5, 0.3, 0.2, 0.5], requires_grad=True)
optimizer = torch.optim.Adam([parm_tensor], lr=0.01)
losses = []

for i in trange(10):
    bt = Backtest(df, Weight_Strategy, cash=cash, commission=.00002, margin=.05)
    stat = bt.run(kwargs=parm_tensor)
    loss = -stat['Equity Final [$]']
    losses.append(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(parm_tensor)

  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'float' object has no attribute 'backward'

沒料 因為他是間接有關聯的問題